In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import statsmodels.formula.api as smf

import scipy.stats as stats

sns.set_theme(style="whitegrid")


## Load Landings Records

In [2]:
landings = pd.read_csv("../data/derived/landings_cleaned.csv", low_memory=False)


In [ ]:
#todo

#gropu by seasonal year, seller_id
#calculate proportion of vessel type for each seller id
#create new var = ice only, water, only, majority ice, majority water

#seasonal year, seller id, ice % (vessel type proportion), thresholded label (ice_dependency)

landings.groupby('seasonal_year', 'seller_id').map()
